### Data Summary
- ZTF DR14 source table
- ZTF DR14 catalog table
- Gaia DR3 catalog table

### Goals for Notebook
- All-sky selection of FG dwafs using LSDB (`main`)
- Run dip detection algorithm using TAPE (`main`)
- Save `.pq` file: 
    - Store ps1_id's 
    - Number of dips, dip infromation
    - Biweight $\sigma_{lc}$ and other Gaia metrics (i.e bp_rp, parallax,...)
 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.units as u
%matplotlib inline
%config InlineBackend.figure_format = "retina"
from matplotlib import rcParams
rcParams['savefig.dpi'] = 550
rcParams['font.size'] = 20
plt.rc('font', family='serif')
import lsdb
import dask
dask.config.set({"temporary-directory" :'/epyc/ssd/users/atzanida/tmp'})

In [2]:
%%time
#Load ZTF, Gaia, and ZTF sources hipscats # note data3/ for epyc
gaia = lsdb.read_hipscat("/epyc/data3/hipscat/test_catalogs/gaia_symbolic")

# load ZTF object table
ztf = lsdb.read_hipscat("/epyc/data3/hipscat/catalogs/ztf_axs/ztf_dr14")

CPU times: user 3.36 s, sys: 435 ms, total: 3.79 s
Wall time: 3.8 s


In [3]:
%%time
# Load ZTF DR14 sources
ztf_sources = lsdb.read_hipscat("/epyc/data3/hipscat/catalogs/ztf_axs/ztf_source")

CPU times: user 50.7 s, sys: 9.49 s, total: 1min
Wall time: 1min


In [4]:
%%time
# crossmatch ZTF + Gaia
_all_sky_object = gaia.crossmatch(ztf).query(
    "nobs_g_ztf_dr14 > 50 and nobs_r_ztf_dr14 > 50 and \
    parallax_gaia > 0 and parallax_over_error_gaia > 5 and \
    bp_rp_gaia > -99 and phot_g_mean_mag_gaia > 0 and \
    teff_gspphot_gaia > 5380 and teff_gspphot_gaia < 7220 and logg_gspphot_gaia > 4.5 \
    and logg_gspphot_gaia < 4.72 and classprob_dsc_combmod_star_gaia > 0.5")

CPU times: user 3.27 s, sys: 72.9 ms, total: 3.34 s
Wall time: 3.35 s


In [5]:
%%time
# Convert crossmatch _sample to lsdb Catalog object
_sample_hips = lsdb.from_dataframe(
    _all_sky_object.compute(), 
    lowest_order=5, 
    highest_order=8, 
    ra_column="ra_gaia", 
    dec_column="dec_gaia", 
    threshold=1_000_000
)